In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:

inputs = keras.Input(shape=(784,), name="digits")
x1 = layers.Dense(64, activation="relu")(inputs)
x2 = layers.Dense(64, activation="relu")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [3]:

# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [4]:

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = model(x_batch_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = loss_fn(y_batch_train, logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * batch_size))



Start of epoch 0
Training loss (for one batch) at step 0: 90.5211
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.8392
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 1.3110
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.9808
Seen so far: 38464 samples

Start of epoch 1
Training loss (for one batch) at step 0: 1.0836
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.8981
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.9956
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.5978
Seen so far: 38464 samples


In [ ]:
import CellCNN
import numpy as np

model = CellCNN.CellCNN((None, 1000, 2), classes=[2, 0], conv=[64,64,16])
ID = CellCNN.InputData((CellCNN.Dataset(1000, center_density=0.01), CellCNN.Dataset(1000, center_density=0.99)), labels=np.array(((0,82.1),(1, 82.1))))
x,y = ID.get_multi_cell_inputs(5000)
model.fit(x,y)

model(x[:10])
y[:10]

import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((x,y))
batch_size = 256
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
